In [1]:
# !pip install -q datasets transformers evaluate

In [2]:
# from datasets import load_dataset

In [3]:
# ds = load_dataset("scene_parse_150", split="train[:50]")

In [4]:
# ds[0]["image"]

In [5]:
# ds = ds.train_test_split(test_size=0.2)

In [6]:
# train_ds = ds["train"]
# test_ds = ds["test"]

In [7]:
import json
from huggingface_hub import cached_download, hf_hub_url

In [8]:
# repo_id = "huggingface/label-files"
# filename = "ade20k-id2label.json"
# id2label = json.load(open(cached_download(hf_hub_url(repo_id, filename, repo_type="dataset")), "r"))
# id2label = {int(k): v for k, v in id2label.items()}
# label2id = {v: k for k, v in id2label.items()}
# num_labels = len(id2label)

In [9]:
id2label={1: 'yes'}

label2id = {'yes': 1}

In [10]:
from transformers import AutoImageProcessor

In [11]:
checkpoint = "nvidia/mit-b0"
image_processor = AutoImageProcessor.from_pretrained(checkpoint, reduce_labels=True)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/opt/conda/envs/al_env/lib/python3.9/site-packages/transformers/models/segformer/image_processing_segformer.py:101: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [12]:
# from torchvision.transforms import ColorJitter

In [13]:
# jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1)

In [14]:
# def train_transforms(example_batch):
#     images = [jitter(x) for x in example_batch["image"]]
#     labels = [x for x in example_batch["annotation"]]
#     inputs = image_processor(images, labels)
#     return inputs

In [15]:
# def val_transforms(example_batch):
#     images = [x for x in example_batch["image"]]
#     labels = [x for x in example_batch["annotation"]]
#     inputs = image_processor(images, labels)
#     return inputs

In [16]:
# train_ds.set_transform(train_transforms)
# test_ds.set_transform(val_transforms)

In [17]:
# type(train_ds[0]['pixel_values'])

In [18]:
import evaluate

In [19]:
metric = evaluate.load("mean_iou")

In [20]:
import numpy as np
import torch
from torch import nn

In [21]:
def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
        metrics = metric.compute(
            predictions=pred_labels,
            references=labels,
            num_labels=num_labels,
            ignore_index=255,
            reduce_labels=False,
        )
        for key, value in metrics.items():
            if type(value) is np.ndarray:
                metrics[key] = value.tolist()
        return metrics

In [22]:
from transformers import AutoModelForSemanticSegmentation, TrainingArguments, Trainer

In [23]:
model = AutoModelForSemanticSegmentation.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.batch_norm.bias', 'decode_head.batch_norm.running_var', 'decode_head.classifier.weight', 'decode_head.batch_norm.weight', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_fuse.weight', 'decode_head.classifier.bias', 'decode_head.batch_norm.running_mean']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
import sys
sys.path.append('../src')
from models import *
from strategies import *
from custom_datasets import *
import numpy as np
import tqdm


import pytorch_lightning as pl
import segmentation_models_pytorch as smp
import os
import torch
import torch.nn as nn
torch.cuda.empty_cache()
from PIL import Image
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset

import pandas as pd

import time
import json

In [25]:
main_path = "/root/Master_Thesis/"
dataframes_path = main_path + "data/dataframes/"
sam_path = main_path + "sam/sam_vit_h_4b8939.pth"
expirements_path = main_path+"expirements/"
# train_df = pd.read_csv(dataframes_path+"train_df.csv")
# test_df = pd.read_csv(dataframes_path+"test_df.csv")
# train_df = train_df[:4096]
# test_df = test_df[:1024]

In [26]:
df = pd.read_csv("/root/Master_Thesis/data/dataframes/brain_df.csv")
df_name = "brain_df"
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, shuffle=True)

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [27]:
len(test_df)

1179

In [28]:
# params = {'n_epoch': 1, 
#                'train_args':{'batch_size': 1, 'num_workers': 0},
#                'test_args':{'batch_size': 1, 'num_workers': 0},
#                'optimizer_args':{'lr': 0.00005, 'momentum': 0.5}}

params = {'n_epoch': 10,
          'train_args':{'batch_size': 8, 'num_workers': 4},
          'test_args':{'batch_size': 8, 'num_workers': 4},
          'optimizer_args':{'lr': 0.005, 'momentum': 0.9},
          'use_sam': False,
          'init_set_size': 20, #int(0.1*len(test_df)),
          'query_num': 4, #int(0.1*len(test_df)),
          'rounds': 10,
          'test_set_size': len(test_df),
          'df': df_name,
          'model': "SegForm"}

In [29]:
if params['use_sam']:
    sam = SAMOracle(checkpoint_path=sam_path)
else:
    sam =None

In [30]:
def get_chorus_data(handler, train_df, test_df):
    return Data(train_df["images"].to_list(), train_df["masks"].to_list(), test_df["images"].to_list(), test_df["masks"].to_list(), handler, df=train_df, path= main_path+"/data/processed/", use_sam=params['use_sam'])

In [31]:
data = get_chorus_data(Handler, train_df, test_df)
data.initialize_labels(params["init_set_size"])

In [32]:
labeled_idxs, labeled_data = data.get_labeled_data()
loader = DataLoader(labeled_data, shuffle=True, **params['train_args'])

20


In [33]:
for batch_idx, (x, y, idxs) in enumerate(loader):
    out = model(x)

In [36]:
out.logits.shape

torch.Size([4, 1, 64, 64])

In [37]:
# preds = nn.functional.interpolate(out.logits,
#                                 size=y.shape[-2:],
#                                 mode="bilinear",
#                                 align_corners=False,
#                             ).argmax(dim=1)

In [40]:
# preds.shape

torch.Size([4, 256, 256])

In [41]:
# preds = preds.type(torch.DoubleTensor)

In [38]:
# print(preds.grad_fn)

In [39]:
# y.size()

In [42]:
from transformers.modeling_outputs import SemanticSegmenterOutput

In [43]:
# if type(out) is SemanticSegmenterOutput:
#     print("True")

In [44]:
net = Net(model, params, device = torch.device("cuda:1"))

In [45]:
strategy = MarginSampling(dataset=data, net=net, sam=sam)

In [46]:
torch.cuda.empty_cache()
logs=[]
print("Round 0")
strategy.train()
logits, mask_gt = strategy.predict(data.get_test_data())
logs.append(f"Round 0 testing metrics: {data.cal_test_metrics(logits, mask_gt )}")
print(logs[0])

for rd in range(1, params["rounds"]):
    print(f"Round {rd}")

    # query
    print("Querying")
    query_idxs = strategy.query(params["query_num"])
    print(query_idxs)

    # update labels
    if params["use_sam"] and rd >= params["activate_sam_at_round"]:
        print("Updating with sam")
        strategy.update(query_idxs, start_sam=True, use_predictor=params["use_predictor"], use_generator=params["use_generator"])
    else:
        print("Updating without sam")
        strategy.update(query_idxs)
    
    print("Reset and train")
    # init_state = torch.load('init_state.pt')
    # strategy.net.net.load_state_dict(init_state)
    strategy.train()

    # calculate accuracy
    logits, maks_gt = strategy.predict(data.get_test_data())
    logs.append(f"Round {rd} testing metrics: {data.cal_test_metrics(logits, mask_gt)}")
    print(logs[rd])
    
params['logs'] = logs

for dirname, _, filenames in os.walk(expirements_path):
    filename = "expirement_{}.json".format(len(filenames))
    file_path = os.path.join(dirname, filename)
    with open(file_path, 'w') as f:
        json.dump(params, f)
        print(filename)

Round 0
20


100%|███████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.21it/s]


Round 0 testing metrics: (tensor(0.), tensor(0.))
Round 1
Querying
[1838 1828 1829 1830]
Updating without sam
Reset and train


RuntimeError: Error(s) in loading state_dict for SegformerForSemanticSegmentation:
	Missing key(s) in state_dict: "segformer.encoder.patch_embeddings.0.proj.weight", "segformer.encoder.patch_embeddings.0.proj.bias", "segformer.encoder.patch_embeddings.0.layer_norm.weight", "segformer.encoder.patch_embeddings.0.layer_norm.bias", "segformer.encoder.patch_embeddings.1.proj.weight", "segformer.encoder.patch_embeddings.1.proj.bias", "segformer.encoder.patch_embeddings.1.layer_norm.weight", "segformer.encoder.patch_embeddings.1.layer_norm.bias", "segformer.encoder.patch_embeddings.2.proj.weight", "segformer.encoder.patch_embeddings.2.proj.bias", "segformer.encoder.patch_embeddings.2.layer_norm.weight", "segformer.encoder.patch_embeddings.2.layer_norm.bias", "segformer.encoder.patch_embeddings.3.proj.weight", "segformer.encoder.patch_embeddings.3.proj.bias", "segformer.encoder.patch_embeddings.3.layer_norm.weight", "segformer.encoder.patch_embeddings.3.layer_norm.bias", "segformer.encoder.blo

In [ ]:
training_args = TrainingArguments(
    output_dir="segformer-b0-scene-parse-150",
    learning_rate=6e-5,
    num_train_epochs=50,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    remove_unused_columns=False
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

In [ ]:
train_ds[0]['pixel_values'].shape

In [ ]:
trainer.train()